In [2]:
# installation required
! pip install langchain

     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     - -------------------------------------- 0.1/1.7 MB 353.1 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.7 MB 327.7 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.7 MB 327.7 kB/s eta 0:00:05
     -- ------------------------------------- 0.1/1.7 MB 297.7 kB/s eta 0:00:06
     -- ------------------------------------- 0.1/1.7 MB 288.1 kB/s eta 0:00:06
     --- ------------------------------------ 0.1/1.7 MB 304.6 kB/s eta 0:00:06
     --- ------------------------------------ 0.1/1.7 MB 304.6 kB/s eta 0:00:06
     --- ------------------------------------ 0.2/1.7 MB 273.3 kB/s eta 0:00:06
     